In [1]:
import numpy as np
import scipy as sp
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
import time
import os
import sys
import pickle
from minibatch import *

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
db_dir = "/home/sweet/2-coursework/725bmes/eeg_deeplearning/data_extraction/src"
ME_db_fname = "mesh_ME_db.pickle"
# MI_db_fname = "prelim_MI_db.pickle"

ME_db = {}
# MI_db = {}


t1 = time.time()
with open(db_dir + "/" + ME_db_fname, "rb") as f:
    ME_db = pickle.load(f)
print("Loaded ME database in %f s" % (time.time()-t1))

np.random.seed(0)
seq_perms = create_rand_seq_permutations(ME_db)
np.random.seed(0)
batch_gen = generate_batch(ME_db, seq_perms, 4)
output_seq, labels = next(batch_gen)
print(output_seq[0][0].shape)

Loaded ME database in 2.444474 s
(10, 11)


In [6]:
def build_conv2d(inputs, filters, kernel_size, strides=(1, 1), padding='same', activation=None, name=None, reuse=None):
    return tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, activation=activation,
                kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), bias_initializer=tf.constant_initializer(0.1), name=name, reuse=reuse)

In [7]:
# def create_CNN_stack(nstack):
#     input_layer = layers.Input(shape=(10,11,1))
#     conv2d_L1 = layers.Conv2D(32, 3, activation="relu", padding="same", 
#                               name="CNN_layer1"+"_"+str(nstack))(input_layer)
#     conv2d_L2 = layers.Conv2D(64, 3, activation="relu", padding="same", 
#                               name="CNN_layer2"+"_"+str(nstack))(conv2d_L1)
#     conv2d_L3 = layers.Conv2D(128, 3, activation="relu", padding="same", 
#                               name="CNN_layer3"+"_"+str(nstack))(conv2d_L2)
#     flat = layers.Flatten()(conv2d_L3)
#     dense = layers.Dense(1024, activation="relu", name="CNN_FC"+"_"+str(nstack))(flat)
#     output_layer = layers.Dropout(dropout_rate, name="CNN_drop"+"_"+str(nstack))(dense)

#     return input_layer, output_layer

# def create_model_cascade():
#     input_layers = []
#     cnn_outputs = []

#     for i in range(S):
#         inp, out = create_CNN_stack(i)
#         input_layers.append(inp)
#         cnn_outputs.append(out)

#     cnn_stack_model = models.Model(inputs=input_layers, outputs=cnn_outputs)

#     concat_CNN_ouputs = layers.concatenate(cnn_stack_model.outputs)
#     concat_CNN_ouputs = layers.Reshape((h_states_lstm, concat_CNN_ouputs.shape[1]//h_states_lstm))(concat_CNN_ouputs)
    
    
#     lstm1 = layers.LSTM(S, return_sequences=True, name="LSTM_1")(concat_CNN_ouputs)
#     lstm2 = layers.LSTM(S, return_sequences=True, dropout=dropout_rate, name="LSTM_2")(lstm1)
#     final_output = layers.Dense(K, activation="softmax", name="FC_softmax")(lstm2[:,:,-1])

#     cascade_model = models.Model(inputs=input_layers, outputs=[final_output])
    
#     return cascade_model

In [21]:
S = 10    # S = number of images
K = 7    # K = number of classes
h_states_lstm = 64
dropout_rate = 0.5
learning_rate = 1e-4
opt = "adam"
lossfn = "categorical_crossentropy"
metric = ["acurracy"]

In [22]:
cascade_model = models.Sequential()
cascade_model.add(layers.Input(shape=(S, 10, 11, 1)))
cascade_model.add(layers.TimeDistributed(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')))
cascade_model.add(layers.TimeDistributed(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')))
cascade_model.add(layers.TimeDistributed(layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')))
cascade_model.add(layers.TimeDistributed(layers.Flatten()))
cascade_model.add(layers.TimeDistributed(layers.Dense(1024, activation='relu')))
cascade_model.add(layers.TimeDistributed(layers.Dropout(0.5)))
cascade_model.add(layers.LSTM(S, return_sequences=True))
cascade_model.add(layers.LSTM(S))
cascade_model.add(layers.Dense(64, activation='relu'))
cascade_model.add(layers.Dropout(0.5))
cascade_model.add(layers.Dense(8, activation='softmax'))
# cascade_model.add(layers.Activation('softmax'))
cascade_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:

# cascade_model = create_model_cascade()
# cascade_model.compile(loss=lossfn, optimizer=opt, metrics=metric)

cascade_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_6 (TimeDist (None, 6, 10, 11, 32)     320       
_________________________________________________________________
time_distributed_7 (TimeDist (None, 6, 10, 11, 64)     18496     
_________________________________________________________________
time_distributed_8 (TimeDist (None, 6, 10, 11, 128)    73856     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 6, 14080)          0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 6, 1024)           14418944  
_________________________________________________________________
time_distributed_11 (TimeDis (None, 6, 1024)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 6, 6)             

In [11]:
from numpy.lib.stride_tricks import as_strided

def sliding_window(a, w = 4, o = 2, copy = False):
    sh = (a.size - w + 1, w)
    st = a.strides * 2
    view = np.lib.stride_tricks.as_strided(a, strides = st, shape = sh)[0::o]
    if copy:
        return view.copy()
    else:
        return view
    


In [ ]:
num_epochs = 100


# output_seq, labels = next(batch_gen)

a = np.arange(769)
b = sliding_window(a, S, S//2)
c = b.flatten()

t1 = time.time()

for epoch in range(num_epochs):
    print("EPOCH %d" %(epoch))
    batch_gen = generate_batch(ME_db, seq_perms, 4)
    for output_seq, labels in batch_gen:
        inputs = np.zeros((len(output_seq), len(c), 10, 11))
        targets = np.zeros((len(output_seq), len(c)))
        stretched = np.zeros((len(c), 10, 11))

        for i in range(len(output_seq)):
            stretched = output_seq[i][c]
            inputs[i] = stretched
            targets[i,:] = labels[i]


        # targets.shape
        inputs = np.expand_dims(inputs, axis=-1)
        # targets = to_categorical(targets, dtype='uint8')
        cascade_model.fit(inputs, to_categorical(labels), batch_size=4)
        
print('Time in total:', time.time()-t1)


EPOCH 0
7/7 [==============================] - 2s 238ms/step - loss: 1.9454 - accuracy: 0.1786


7/7 [==============================] - 2s 235ms/step - loss: 1.9538 - accuracy: 0.0714


7/7 [==============================] - 2s 232ms/step - loss: 1.9395 - accuracy: 0.1786
EPOCH 1
7/7 [==============================] - 2s 237ms/step - loss: 1.9564 - accuracy: 0.0000e+00


7/7 [==============================] - 2s 235ms/step - loss: 1.9465 - accuracy: 0.2500


7/7 [==============================] - 2s 236ms/step - loss: 1.9494 - accuracy: 0.1071
EPOCH 2
7/7 [==============================] - 2s 237ms/step - loss: 1.9482 - accuracy: 0.0357


7/7 [==============================] - 2s 234ms/step - loss: 1.9688 - accuracy: 0.0357


3/7 [===========>..................] - ETA: 0s - loss: 1.9376 - accuracy: 0.3333

In [13]:
# inputs1 = layers.Input(shape=(3, 1))
# lstm1, state_h, state_c = layers.LSTM(1, return_sequences=True, return_state=True)(inputs1)
# model = models.Model(inputs=inputs1, outputs=[state_h])
# # define input data
# data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))
# # make and show prediction
# print(model.predict(data))